In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import sys
from selenium.webdriver.chrome.options import Options
import os

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: None]


In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# df_watched

In [56]:
def date_parser(date):
    return pd.to_datetime(date).date()

In [57]:
df_watched = pd.read_csv('letterboxd_data/watched.csv', parse_dates=['Date'], date_parser=date_parser)
df_watched.rename(columns={'Letterboxd URI':'URL', 'Date':'Date_Entered'},inplace=True)
df_watched.head()

,Date_Entered,Name,Year,URL
0,2020-04-11,Parasite,2019,https://boxd.it/hTha
1,2020-04-11,Knives Out,2019,https://boxd.it/jWEA
2,2020-04-11,The Social Network,2010,https://boxd.it/17ue
3,2020-04-11,Once Upon a Time… in Hollywood,2019,https://boxd.it/gH0O
4,2020-04-11,Joker,2019,https://boxd.it/h4cS


In [5]:
df_watched['URL'].is_unique

True

In [6]:
len(df_watched)

726

# df_ratings

In [58]:
df_ratings = pd.read_csv('letterboxd_data/ratings.csv', parse_dates=['Date'], date_parser=date_parser)
df_ratings.rename(columns={'Letterboxd URI':'URL', 'Date':'Date_Rated'},inplace=True)
df_ratings.drop(columns=['Name', 'Year'],inplace=True)
df_ratings.head()

,Date_Rated,URL,Rating
0,2022-09-24,https://boxd.it/a5fa,5.0
1,2022-09-24,https://boxd.it/7bQA,5.0
2,2022-09-24,https://boxd.it/29FA,5.0
3,2022-09-24,https://boxd.it/29EW,5.0
4,2022-09-24,https://boxd.it/5RFA,5.0


# df_complete

In [59]:
df_complete = df_watched.merge(df_ratings, on='URL', how='left')
display(df_complete.tail())

,Date_Entered,Name,Year,URL,Date_Rated,Rating
721,2024-02-22,Lethal Seduction,2015,https://boxd.it/bGVO,2024-03-01,1.0
722,2024-02-22,The House of Yes,1997,https://boxd.it/1fGM,2024-02-22,3.5
723,2024-02-24,The Dictator,2012,https://boxd.it/2DEW,2024-02-24,3.0
724,2024-02-28,Sanctuary,2022,https://boxd.it/wPKW,2024-02-28,4.0
725,2024-03-01,Stoker,2013,https://boxd.it/34S2,2024-03-01,2.0


In [60]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 0 to 725
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date_Entered  726 non-null    datetime64[ns]
 1   Name          726 non-null    object        
 2   Year          726 non-null    int64         
 3   URL           726 non-null    object        
 4   Date_Rated    117 non-null    datetime64[ns]
 5   Rating        117 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 39.7+ KB


In [61]:
df_complete['Decade'] = df_complete['Year'] - (df_complete['Year']%10)
df_complete.head()

,Date_Entered,Name,Year,URL,Date_Rated,Rating,Decade
0,2020-04-11,Parasite,2019,https://boxd.it/hTha,2023-01-04,5.0,2010
1,2020-04-11,Knives Out,2019,https://boxd.it/jWEA,NaT,NaN,2010
2,2020-04-11,The Social Network,2010,https://boxd.it/17ue,2023-12-25,4.5,2010
3,2020-04-11,Once Upon a Time… in Hollywood,2019,https://boxd.it/gH0O,NaT,NaN,2010
4,2020-04-11,Joker,2019,https://boxd.it/h4cS,NaT,NaN,2010


In [62]:
df_complete.tail()

,Date_Entered,Name,Year,URL,Date_Rated,Rating,Decade
721,2024-02-22,Lethal Seduction,2015,https://boxd.it/bGVO,2024-03-01,1.0,2010
722,2024-02-22,The House of Yes,1997,https://boxd.it/1fGM,2024-02-22,3.5,1990
723,2024-02-24,The Dictator,2012,https://boxd.it/2DEW,2024-02-24,3.0,2010
724,2024-02-28,Sanctuary,2022,https://boxd.it/wPKW,2024-02-28,4.0,2020
725,2024-03-01,Stoker,2013,https://boxd.it/34S2,2024-03-01,2.0,2010


In [63]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 0 to 725
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date_Entered  726 non-null    datetime64[ns]
 1   Name          726 non-null    object        
 2   Year          726 non-null    int64         
 3   URL           726 non-null    object        
 4   Date_Rated    117 non-null    datetime64[ns]
 5   Rating        117 non-null    float64       
 6   Decade        726 non-null    int64         
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 45.4+ KB


# Web Scrapping Function

In [64]:
def name_val(i,table):
    vals = []
    name = table.find_all('div')[i]
    for val in name.find_all('a'):
        vals.append(val.text)
    return vals

def name_val_themes(i,table):
    vals = []
    # name = table.find_all('div')[i]
    # for val in name.find_all('a'):
    #     if val.text == 'Show All…':
    #         new_theme_page = requests.get(f"{URL}"+f"{val.attrs['href']}").text
    #         section = new_theme_page.find('div').text
    #         sel = Selector(text=new_theme_page)
    #         print(section)
    #         if (sel):
    #             print(1)

#             driver = webdriver.Chrome(service=service)
#             new_url = f"{URL}"+f"{val.attrs['href']}"
#             driver.get(new_url)
#             try:
#                 WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//body/div/div/div/section/section/h2/a/span[@class='label']")))
#                 temp = driver.find_elements_by_xpath("//body/div/div/div/section/section/h2/a/span[@class='label']")
#                 for element in temp:
#                     vals.append(element.text)
#             except StaleElementReferenceException:
#                 print("Element became stale, handle accordingly")
#             except Exception as e:
#                 print(f"An error occurred: {e}")
#                 vals = np.nan

#             sel = Selector(text=new_theme_page)
#             for new_val in new_theme_page.find_all('h2', attrs={'class':'title'}):
#             print((sel.xpath("//title/text()").extract()))
#                 print(new_val.text)
#                 vals.append(new_val.text)
    # print(vals)
    return vals

def web_scrapping(df, ex_data):
    error_urls = []
    for row in df.itertuples():
        URL = row.URL
        page = requests.get(f"{URL}").text
        if page:
            print(row.Name)

        soup = BeautifulSoup(page,"html.parser")

        Watched_By = ''
        Avg_Rating = ''

        # service = Service(executable_path="/content/drive/MyDrive/Colab Notebooks/chromedriverlinux")
        driver = webdriver.Chrome(options=chrome_options)
        try:
            driver.get(URL)
            try:
                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//span[@class='average-rating']/a")))
                Avg_Rating = driver.find_element(By.XPATH, "//span[@class='average-rating']/a").get_attribute('data-original-title')
            except StaleElementReferenceException:
                print("Element became stale, handle accordingly")
            except Exception as e:
                print(f"An error occurred: {e}")
                error_urls.append(URL)
                Avg_Rating = np.nan

            try:
                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//div[@id='content']/div[@class='content-wrap']/div/div/section/ul/li[@class='stat filmstat-watches']/a")))
                Watched_By = driver.find_element(By.XPATH, "//div[@id='content']/div[@class='content-wrap']/div/div/section/ul/li[@class='stat filmstat-watches']/a").get_attribute('data-original-title')
            except StaleElementReferenceException:
                print("Element became stale, handle accordingly")
            except Exception as e:
                print(f"An error occurred: {e}")
                error_urls.append(URL)
                Watched_By = np.nan

        except:
            error_urls.append(URL)

        finally:
            print(Watched_By, Avg_Rating)
            driver.quit()



        try:
            Synopsis = str(soup.find('div',attrs={'class':'review body-text -prose -hero prettify'}).find('div').text).strip()
        except:
            Synopsis = np.nan

        try:
            r = soup.find('p',attrs={'class':'text-link text-footer'}).text
            runtime_extract = re.search("[0-9]{2,3}",r)
            Runtime = int(r[runtime_extract.span()[0]:runtime_extract.span()[1]])
        except:
            Runtime = np.nan

        Cast = []
        try:
            cast = soup.find('div', attrs={'id':'tab-cast'})
            for a in cast.find_all('a'):
                if a.text == 'Show All…':
                    pass
                else:
                    Cast.append(a.text)
        except:
            pass

        Director = []
        Co_Director = []
        Asst_Director = []
        Add_Directing = []
        Producer = []
        Exec_Producer = []
        Writer = []
        Original_Writer = []
        Casting = []
        Editor = []
        Cinematographer = []
        Camera_Operator = []
        Lighting = []
        Add_Photography = []
        Production_Design = []
        Art_Direction = []
        Set_Decoration = []
        Special_Effect = []
        Visual_Effect = []
        Title_Design = []
        Stunt = []
        Choreography = []
        Composer = []
        Song = []
        Sound = []
        Costume_Design = []
        Makeup = []
        Hair_Styling = []

        try:
            crew = soup.find('div',attrs={'id':'tab-crew'})
            for i,h in enumerate(crew.find_all('h3')):
                crew_title = h.find('span',attrs={'class':'crewrole -short'}).text
                if crew_title in ['Director','Directors']:
                    Director = name_val(i,crew)
                elif crew_title in ['Co-Director','Co-Directors']:
                    Co_Director = name_val(i,crew)
                elif crew_title in ['Asst. Director', 'Asst. Directors']:
                    Asst_Director = name_val(i,crew)
                elif crew_title in ['Add. Directing','Add. Directings']:
                    Add_Directing = name_val(i,crew)
                elif crew_title in ['Producer','Producers']:
                    Producer = name_val(i,crew)
                elif crew_title in ['Exec. Producer','Exec. Producers']:
                    Exec_Producer = name_val(i,crew)
                elif crew_title in ['Writer','Writers']:
                    Writer = name_val(i,crew)
                elif crew_title in ['Original Writer','Original Writers']:
                    Original_Writer = name_val(i,crew)
                elif crew_title in ['Casting','Castings']:
                    Casting = name_val(i,crew)
                elif crew_title in ['Editor','Editors']:
                    Editor = name_val(i,crew)
                elif crew_title in ['Cinematography']:
                    Cinematographer = name_val(i,crew)
                elif crew_title in ['Camera Operator','Camera Operators']:
                    Camera_Operator = name_val(i,crew)
                elif crew_title in ['Lighting','Lightings']:
                    Lighting = name_val(i,crew)
                elif crew_title in ['Add. Photography','Add. Photographies']:
                    Add_Photography = name_val(i,crew)
                elif crew_title in ['Production Design','Producton Designs']:
                    Production_Design = name_val(i,crew)
                elif crew_title in ['Art Direction','Art Directions']:
                    Art_Direction = name_val(i,crew)
                elif crew_title in ['Set Decoration','Set Decorations']:
                    Set_Decoration = name_val(i,crew)
                elif crew_title in ['Special Effect','Special Effects']:
                    Special_Effect = name_val(i,crew)
                elif crew_title in ['Visual Effect','Visual Effects']:
                    Visual_Effect = name_val(i,crew)
                elif crew_title in ['Title Design','Title Designs']:
                    Title_Design = name_val(i,crew)
                elif crew_title in ['Stunt','Stunts']:
                    Stunt = name_val(i,crew)
                elif crew_title in ['Choreography','Choreographies']:
                    Choreography = name_val(i,crew)
                elif crew_title in ['Composer','Composers']:
                    Composer = name_val(i,crew)
                elif crew_title in ['Song','Songs']:
                    Song = name_val(i,crew)
                elif crew_title in ['Sound','Sounds']:
                    Sound = name_val(i,crew)
                elif crew_title in ['Costume Design','Costume Designs']:
                    Costume_Design = name_val(i,crew)
                elif crew_title in ['Makeup','Makeups']:
                    Makeup = name_val(i,crew)
                elif crew_title in ['Hairstyling','Hairstylings']:
                    Hair_Styling = name_val(i,crew)
        except:
            pass

        Studio = []
        Country = []
        Language = []
        Alternative_Title = []

        try:
            details = soup.find('div',attrs={'id':'tab-details'})
            for i,h in enumerate(details.find_all('h3')):
                detail_title = h.find('span').text
                if detail_title in ['Studio','Studios']:
                    Studio = name_val(i,details)
                elif detail_title in ['Country','Countries']:
                    Country = name_val(i,details)
                elif detail_title in ['Language','Languages','Original Language', 'Spoken Languages']:
                    Language = name_val(i,details)
                elif detail_title in ['Alternative Title','Alternative Titles']:
                    Alternative_Title = str(details.find_all('div')[i].find('p').text).strip()
        except:
            pass

        Genre = []
        Theme = []

        try:
            genres = soup.find('div',attrs={'id':'tab-genres'})
            for i,h in enumerate(genres.find_all('h3')):
                genre_title = h.find('span').text
                if genre_title in ['Genre','Genres']:
                    Genre = name_val(i,genres)
                elif genre_title in ['Theme','Themes']:
                    Theme = name_val_themes(i,genres)
        except:
            pass

        columns = ['Name', 'Year', 'Decade', 'Rating', 'Date_Entered', 'Date_Rated','Avg_Rating', 'Watched_By','URL', 'Synopsis','Runtime','Cast','Director','Co_Director','Asst_Director','Add_Directing','Producer',
                   'Exec_Producer','Writer','Original_Writer','Casting','Editor','Cinematographer','Camera_Operator','Lighting',
                   'Add_Photography','Production_Design','Art_Direction','Set_Decoration','Special_Effect','Visual_Effect',
                   'Title_Design','Stunt','Choreography','Composer','Song','Sound','Costume_Design','Makeup','Hair_Styling',
                   'Studio','Country','Language','Alt_Title','Genre','Theme']
        data = [[row.Name,row.Year,row.Decade,row.Rating,row.Date_Entered,row.Date_Rated,Avg_Rating,Watched_By,URL,Synopsis,Runtime,Cast,Director,Co_Director,Asst_Director,Add_Directing,Producer,Exec_Producer,Writer,
                 Original_Writer,Casting,Editor,Cinematographer,Camera_Operator,Lighting,Add_Photography,Production_Design,
                 Art_Direction,Set_Decoration,Special_Effect,Visual_Effect,Title_Design,Stunt,Choreography,Composer,Song,Sound,
                 Costume_Design,Makeup,Hair_Styling,Studio,Country,Language,Alternative_Title,Genre,Theme]]
        df = pd.DataFrame(data,columns=columns)
        ex_data = pd.concat([ex_data,df], ignore_index=True)
    return ex_data, error_urls

# Scrapping Process

In [47]:
file_path = "movie_data.csv"
if os.path.exists(file_path):
    print("if true")
    movie_data = pd.read_csv('movie_data.csv', index_col=0)
    print(len(movie_data))
    ex_data = movie_data[movie_data['URL'].isin(df_complete['URL'])]
    print(len(ex_data))
    df_to_extract = df_complete[~df_complete['URL'].isin(ex_data['URL'])]
    columns = ['Name', 'Year', 'Decade', 'Rating', 'Date_Entered', 'Date_Rated', 'Avg_Rating', 'Watched_By', 'URL', 'Synopsis','Runtime','Cast','Director','Co_Director','Asst_Director','Add_Directing','Producer',
            'Exec_Producer','Writer','Original_Writer','Casting','Editor','Cinematographer','Camera_Operator','Lighting',
            'Add_Photography','Production_Design','Art_Direction','Set_Decoration','Special_Effect','Visual_Effect',
            'Title_Design','Stunt','Choreography','Composer','Song','Sound','Costume_Design','Makeup','Hair_Styling',
            'Studio','Country','Language','Alt_Title','Genre','Theme']
    print(len(df_to_extract))
    new_ex_data = pd.DataFrame(columns=columns)
    new_ex_data, error_urls = web_scrapping(df_to_extract, new_ex_data)
    print(len(new_ex_data))
    error_urls_df = new_ex_data[(new_ex_data['URL'].isin(error_urls)) & (new_ex_data['Watched_By'].str.len()==0)]
    print(len(error_urls_df))
    while (len(error_urls_df)!=0):
        display(error_urls_df)
        new_ex_data, error_urls = web_scrapping(error_urls_df, new_ex_data)
        error_urls_df = new_ex_data[(new_ex_data['URL'].isin(error_urls)) & (new_ex_data['Watched_By'].str.len()==0)]
else:
    print("if false")
    columns = ['Name', 'Year', 'Decade', 'Rating', 'Date_Entered', 'Date_Rated', 'Avg_Rating', 'Watched_By', 'URL', 'Synopsis','Runtime','Cast','Director','Co_Director','Asst_Director','Add_Directing','Producer',
            'Exec_Producer','Writer','Original_Writer','Casting','Editor','Cinematographer','Camera_Operator','Lighting',
            'Add_Photography','Production_Design','Art_Direction','Set_Decoration','Special_Effect','Visual_Effect',
            'Title_Design','Stunt','Choreography','Composer','Song','Sound','Costume_Design','Makeup','Hair_Styling',
            'Studio','Country','Language','Alt_Title','Genre','Theme']
    ex_data = pd.DataFrame(columns=columns)
    new_ex_data = pd.DataFrame(columns=columns)
    new_ex_data, error_urls = web_scrapping(df_complete, new_ex_data)
    error_urls_df = new_ex_data[(new_ex_data['URL'].isin(error_urls)) & (new_ex_data['Watched_By'].str.len()==0)]
    while (len(error_urls_df)!=0):
        new_ex_data, error_urls = web_scrapping(error_urls_df, new_ex_data)
        error_urls_df = new_ex_data[(new_ex_data['URL'].isin(error_urls)) & (new_ex_data['Watched_By'].str.len()==0)]
new_ex_data = new_ex_data[new_ex_data['Watched_By'].str.len()!=0]
new_ex_data['Year'] = new_ex_data['Year'].astype('int')
new_ex_data['Decade'] = new_ex_data['Decade'].astype('int')
new_ex_data['Avg_Rating'] = new_ex_data['Avg_Rating'].str.replace(',','').str.extract(r'(\d+\.\d+|\d+)').astype('float')
new_ex_data['Watched_By'] = new_ex_data['Watched_By'].str.replace(',','').str.extract(r'(\d+)').astype('int')
new_ex_data['Runtime'] = new_ex_data['Runtime'].astype('float')
new_ex_data['Diff_from_Avg_Rating'] = new_ex_data['Rating'] - new_ex_data['Avg_Rating']
ex_data = pd.concat([ex_data, new_ex_data], ignore_index=True)
movie_data = pd.concat([movie_data, new_ex_data], ignore_index=True)
display(len(ex_data))
display(len(movie_data))
ex_data.to_csv('complete_data.csv')
movie_data.to_csv('movie_data.csv')

if true
Redline
Watched by 62,641 members Weighted average of 4.03 based on 43,071 ratings
The Toy Warrior
An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61EC9AD22+56930]
	(No symbol) [0x00007FF61EC0F622]
	(No symbol) [0x00007FF61EAC42E5]
	(No symbol) [0x00007FF61EB098ED]
	(No symbol) [0x00007FF61EB09A2C]
	(No symbol) [0x00007FF61EB4A967]
	(No symbol) [0x00007FF61EB2BCDF]
	(No symbol) [0x00007FF61EB481E2]
	(No symbol) [0x00007FF61EB2BA43]
	(No symbol) [0x00007FF61EAFD438]
	(No symbol) [0x00007FF61EAFE4D1]
	GetHandleVerifier [0x00007FF61F016AAD+3709933]
	GetHandleVerifier [0x00007FF61F06FFED+4075821]
	GetHandleVerifier [0x00007FF61F06817F+4043455]
	GetHandleVerifier [0x00007FF61ED39756+706710]
	(No symbol) [0x00007FF61EC1B8FF]
	(No symbol) [0x00007FF61EC16AE4]
	(No symbol) [0x00007FF61EC16C3C]
	(No symbol) [0x00007FF61EC068F4]
	BaseThreadInitThunk [0x00007FFB0A5C7344+20]
	RtlUserThreadStart [0x00007FFB0B7426B1+33]

Watched by 138 members nan
Tom and Jerry Meet She

726

726

In [49]:
im_data = pd.read_csv('complete_data.csv', index_col=0)
im_data.tail()

,Name,Year,Decade,Rating,Date_Entered,Date_Rated,Avg_Rating,Watched_By,URL,Synopsis,...,Costume_Design,Makeup,Hair_Styling,Studio,Country,Language,Alt_Title,Genre,Theme,Diff_from_Avg_Rating
721,Lethal Seduction,2015,2010,1.0,2024-02-22 00:00:00,2024-03-01 00:00:00,2.87,395,https://boxd.it/bGVO,"High School senior Mark Richards has never minded his overprotective widowed mother, Tanya, and is a good son to her as he prepares to go off to Princeton in the fall. However, when he comes under the sexual spell of the rapacious, manipulative older woman Carissa Barrington, he finds himself in the middle of two strong, unreasonable women--one of whom is insane...",...,['Martha Gretsch'],[],[],['Indy Entertainment'],['USA'],"['English', 'French']","Innocence volée, Stripped of Innocence, Atracció mortal, Posedlá láskou, Atracción mortal, Seduzione letale, Sedução letal, Смертельное Искушение, Usodno zapeljevanje","['Thriller', 'TV Movie']",[],-1.87
722,The House of Yes,1997,1990,3.5,2024-02-22 00:00:00,2024-02-22 00:00:00,3.41,14205,https://boxd.it/1fGM,"Jackie-O is anxiously awaiting the visit of her brother home for Thanksgiving, but isn't expecting him to bring a friend — and she's even more shocked to learn that this friend is his fiance. It soon becomes clear that her obsession with Jackie Kennedy is nothing compared to her obsession with her brother, and she isn't the only member of the family with problems.",...,['Edi Giguere'],['Jason Rail'],['Jason Rail'],"['Miramax', 'Bandeira Entertainment', 'Spelling Films']",[],['English'],"La casa del sì, Дом, где говорят Да, Almas gemelas, Wer hat Angst vor Jackie-O.?, Upiorne święto, Yes之屋, A Casa do Sim, 더 하우스 오브 예스","['Comedy', 'Drama']",[],0.09
723,The Dictator,2012,2010,3.0,2024-02-24 00:00:00,2024-02-24 00:00:00,2.82,410636,https://boxd.it/2DEW,The heroic story of a dictator who risks his life to ensure that democracy would never come to the country he so lovingly oppressed.,...,"['Jeffrey Kurland', 'Jason Alper']","['Marjorie Durand', 'Judy Chin', 'Naomi Donne', 'Diana Choi', 'Roland Blancaflor', 'Mike Marino']","['Erika Abberton', 'Christine Fennell', 'Naomi Donne', 'Jasen Joseph Sica']","['Paramount', 'Four by Two', 'Berg Mandel Schaffer Productions', 'Scott Rudin Productions']","['UK', 'USA']","['English', 'Hebrew\xa0(modern)']","O Ditador, A diktátor, Dictatorul, Diktaattori, El dictador, Il dittatore, O diktatoras, Диктатор, Ο δικτάτορας, DICTATOR_THE_2012, Le Dictateur, Der Diktator, Diktatör, Le dictateur, Diktatoren, 独裁者, Diktátor, הדיקטטור, จอมเผด็จการ, Dyktator, Диктаторът, Diktator, Ο Δικτάτορας, 독재자, El Dictador, Diktators, ディクテーター 身元不明でニューヨーク, 大鈍裁者, 大獨裁者落難記, Kẻ Độc Tài, დიქტატორი",['Comedy'],[],0.18
724,Sanctuary,2022,2020,4.0,2024-02-28 00:00:00,2024-02-28 00:00:00,3.45,56295,https://boxd.it/wPKW,"Confined to a claustrophobic hotel room, the heir to a hotel empire and the dominatrix who has primed him for success become locked in a battle of wits and wills as he tries to end his relationship with her.",...,['Mirren Gordon-Crozier'],['Mary Chipman'],['Lauren Citera'],"['Rumble Films', 'Charades', 'Mosaic', 'Hype Studios']","['France', 'USA']",['English'],"Šventovė, El templo, 생츄어리, 피난처, Soumission, Стоп-слово, Sanctuary - Lui fa il gioco. Lei fa le regole., 圣所, Укриття, Santuario, Santuário, 聖地, Sanctuaire, Zavetišče, Sığınak",['Thriller'],[],0.55
725,Stoker,2013,2010,2.0,2024-03-01 00:00:00,2024-03-01 00:00:00,3.60,172869,https://boxd.it/34S2,"After India's father dies, her Uncle Charlie, who she never knew existed, comes to live with her and her unstable mother. She comes to suspect this mysterious, charming man has ulterior motives and becomes increasingly infatuated with him.",...,"['Kurt Swanson', 'Bart Mueller']","['Gloria Belz', 'Meredith Johns', 'Kyra Panchenko', 'Linda Boykin-Williams', 'Sandy Jo Johnston', 'Jennifer Albada', 'Carol Frazier', 'Georgia Dunn']","['Candace Neal', 'Michealle Vanderpool', 'Melinda Dunn', 'Jody Sunday', 'Abbey Bradley', 'Carlita Ma

In [50]:
im_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 0 to 725
Data columns (total 47 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  726 non-null    object 
 1   Year                  726 non-null    int64  
 2   Decade                726 non-null    int64  
 3   Rating                117 non-null    float64
 4   Date_Entered          726 non-null    object 
 5   Date_Rated            117 non-null    object 
 6   Avg_Rating            721 non-null    float64
 7   Watched_By            726 non-null    int64  
 8   URL                   726 non-null    object 
 9   Synopsis              726 non-null    object 
 10  Runtime               726 non-null    float64
 11  Cast                  726 non-null    object 
 12  Director              726 non-null    object 
 13  Co_Director           726 non-null    object 
 14  Asst_Director         726 non-null    object 
 15  Add_Directing         7

In [51]:
def dim_tables(df, column_name, column_index):
    column_dict = {}
    for row in df.itertuples():
        column = row[column_index]
        for item in column:
            if item in column_dict.keys():
                column_dict[item][0] += 1
                if not(np.isnan(row.Rating)):
                    column_dict[item][1] += row.Rating
                    column_dict[item][2] += 1
            else:
                if not(np.isnan(row.Rating)):
                    column_dict[item] = [1, row.Rating, 1]
                else: 
                    column_dict[item] = [1, 0, 0]
    column_df = pd.DataFrame.from_dict(column_dict, orient='index', columns=['Count', 'Rating', 'Rating_Count'])
    column_df.reset_index(inplace=True)
    column_df.columns = [column_name, 'Count', 'Rating', 'Rating_Count']
    column_df['My_Rating_Avg'] = np.where(column_df['Rating_Count']!=0, column_df['Rating']/column_df['Rating_Count'], np.nan)
    column_df.sort_values('Count', inplace=True, ascending=False)
    column_df.reset_index(drop=True, inplace=True)
    column_df = column_df[column_df[column_name].str.len()!=0]
    return column_df

def dim_files(df, list_columns_idx):
    for column, idx in list_columns_idx:
        dim_df = dim_tables(df, column, idx)
        display(dim_df.head())
        dim_df.to_csv(column + '.csv')
        
    

def convert_str_to_list(string):
    return string.strip('[]').split(', ')

def convert_columns_to_list(df, list_columns_idx, convert_str_to_list):
    for column in list_columns_idx:
        print(column[0])
        df[column[0]] = df[column[0]].apply(convert_str_to_list)

In [52]:
list_columns_idx = [['Cast', 12], ['Director', 13], ['Co_Director', 14], ['Asst_Director', 15], ['Add_Directing', 16], 
                    ['Producer', 17], ['Exec_Producer', 18], ['Writer', 19], ['Original_Writer', 20], ['Casting', 21],
                    ['Editor', 22], ['Cinematographer', 23], ['Camera_Operator', 24], ['Lighting', 25], ['Add_Photography', 26],
                    ['Production_Design', 27], ['Art_Direction', 28], ['Set_Decoration', 29], ['Special_Effect', 30],
                    ['Visual_Effect', 31], ['Title_Design', 32], ['Stunt', 33], ['Choreography', 34], ['Composer', 35],
                    ['Song', 36], ['Sound', 37], ['Costume_Design', 38], ['Makeup', 39], ['Hair_Styling', 40], ['Studio', 41],
                    ['Country', 42], ['Language', 43], ['Genre', 45], ['Theme', 46]]
print(len(list_columns_idx))

34


In [53]:
convert_columns_to_list(im_data, list_columns_idx, convert_str_to_list)

Cast
Director
Co_Director
Asst_Director
Add_Directing
Producer
Exec_Producer
Writer
Original_Writer
Casting
Editor
Cinematographer
Camera_Operator
Lighting
Add_Photography
Production_Design
Art_Direction
Set_Decoration
Special_Effect
Visual_Effect
Title_Design
Stunt
Choreography
Composer
Song
Sound
Costume_Design
Makeup
Hair_Styling
Studio
Country
Language
Genre
Theme


In [54]:
dim_files(im_data, list_columns_idx)

,Cast,Count,Rating,Rating_Count,My_Rating_Avg
0,'Shah Rukh Khan',46,11.5,3,3.833333
1,'Akshay Kumar',37,0.0,0,NaN
2,'Salman Khan',34,0.0,0,NaN
3,'Stan Lee',28,0.0,0,NaN
4,'Anupam Kher',28,0.0,0,NaN


,Director,Count,Rating,Rating_Count,My_Rating_Avg
0,'Priyadarshan',12,0.0,0,NaN
1,'Christopher Nolan',11,26.0,6,4.333333
2,'David Fincher',8,12.5,3,4.166667
3,'Quentin Tarantino',8,4.5,1,4.500000
4,'Anees Bazmee',7,0.0,0,NaN


,Co_Director,Count,Rating,Rating_Count,My_Rating_Avg
1,'Akiyuki Simbo',3,12.0,3,4.0
2,'Lee Unkrich',3,4.5,1,4.5
3,'Bob Peterson',1,0.0,0,NaN
4,'Adesh Prasad',1,0.0,0,NaN
5,'Mike Thurmeier',1,0.0,0,NaN


,Asst_Director,Count,Rating,Rating_Count,My_Rating_Avg
1,'Matthew Sharp',12,4.5,1,4.50
2,'Lars P. Winther',10,0.0,0,NaN
3,'Jamie Christopher',10,0.0,0,NaN
4,'Chris Carreras',8,0.0,0,NaN
5,'Nilo Otero',7,17.0,4,4.25


,Add_Directing,Count,Rating,Rating_Count,My_Rating_Avg
1,'Alexander Witt',11,0.0,0,NaN
2,'Darrin Prescott',7,0.0,0,NaN
3,'John Mahaffie',6,5.0,1,5.0
4,'Dan Bradley',6,0.0,0,NaN
5,'Brian Smrz',5,0.0,0,NaN


...

,Studio,Count,Rating,Rating_Count,My_Rating_Avg
0,'Warner Bros. Pictures',50,39.0,10,3.900000
1,'Yash Raj Films',35,4.5,1,4.500000
2,'20th Century Fox',33,10.5,3,3.500000
3,'Paramount',31,12.5,3,4.166667
4,'Columbia Pictures',29,24.0,6,4.000000


,Country,Count,Rating,Rating_Count,My_Rating_Avg
0,'USA',388,248.0,64,3.875000
1,'India',263,31.0,9,3.444444
2,'UK',75,58.0,16,3.625000
3,'Japan',48,129.5,34,3.808824
4,'Canada',23,13.0,3,4.333333


,Language,Count,Rating,Rating_Count,My_Rating_Avg
0,'English',452,286.0,73,3.917808
1,'Hindi',259,35.5,10,3.550000
2,'French',66,29.0,8,3.625000
3,'Japanese',60,139.5,37,3.770270
4,'Spanish',53,54.5,14,3.892857


,Genre,Count,Rating,Rating_Count,My_Rating_Avg
0,'Drama',314,230.5,57,4.043860
1,'Action',260,111.5,30,3.716667
2,'Comedy',228,114.0,31,3.677419
3,'Adventure',190,65.0,17,3.823529
4,'Thriller',178,119.5,32,3.734375


,Theme,Count,Rating,Rating_Count,My_Rating_Avg


In [55]:
df_diary = pd.read_csv('letterboxd_data/diary.csv', parse_dates=['Watched Date', 'Date'])
display(df_diary)
df_diary.rename(columns={'Letterboxd URI':'URL', 'Date':'Date_Diary', 'Watched Date':'Date_Watched'},inplace=True)
df_diary['Year'] = df_diary['Date_Watched'].dt.year
df_diary.to_csv('Diary.csv')
def longest_streak_w(df, idx_date_column):
    df = df.set_index(idx_date_column)
    df.sort_index(inplace=True)
    weekly_counts = df.resample('W').size()
    streak = 0
    longest_streak = 0
    for count in weekly_counts:
        if count > 0:
            streak += 1
            longest_streak = max(longest_streak, streak)
        else:
            streak = 0
    return longest_streak

Longest_Streak_Weekly = pd.DataFrame({'Longest_Streak_Weekly':longest_streak_w(df_diary, 'Date_Watched')},index=[0])
Longest_Streak_Weekly.to_csv('Longest_Streak_Weekly.csv')
display(Longest_Streak_Weekly)

two_plus_days_df = df_diary.groupby('Date_Watched', as_index=False).size()
two_plus_days_df = two_plus_days_df[two_plus_days_df['size']>1]
two_plus_days_df.to_csv('two_plus_days_df.csv')
display(two_plus_days_df)